In [19]:
import pymysql
import pandas as pd
import numpy as np
import time

In [20]:
#手机支付电话号码
sql1 = """select oo.user_id,vo.user_id,oo.mobile 手机号,FROM_UNIXTIME(oo.pay_time),vo.unionid
from  `kkb-cloud-vipcourse`.vip_order vo
LEFT JOIN `kkb-cloud-order`.order_info oo 

on vo.openid=oo.openid and ABS(vo.pay_time-oo.pay_time)<100
where  vo.course_id = 76 and oo.mobile != ''
and vo.`status`=2 
and  FROM_UNIXTIME(vo.pay_time,'%Y-%m-%d %H:%i:S') BETWEEN '2020-05-26 16:00:00' and '2020-05-27 16:00:00'#支付电话"""

In [21]:
#每日开课电话号码
sql2 = """select oo.user_id,vo.user_id,oo.mobile 手机号,FROM_UNIXTIME(oo.pay_time),vo.unionid
from  `kkb-cloud-vipcourse`.vip_order vo
LEFT JOIN `kkb-cloud-order`.order_info oo 

on vo.openid=oo.openid and ABS(vo.pay_time-oo.pay_time)<100
where  vo.course_id = 76 and oo.mobile != ''
and vo.`status`=2 

and vo.user_id in (SELECT user_id
FROM `kkb-cloud-vipcourse`.vip_order 
WHERE course_id = 76 AND `status` = 2 AND class_plan_id in
(SELECT id FROM
`kkb-cloud-vipcourse`.vip_seller_class_plan
WHERE from_unixtime((class_time/1000),'%Y-%m-%d %H:%i:S') >'2020-05-27 00:00:00' 
AND `status` = 2))
"""

In [22]:
con=pymysql.connect("rr-2ze4j8i5pm6q3u4s4jo.mysql.rds.aliyuncs.com","lirenqiang","5s2DS5n9LdWpUYWN")

In [23]:
data1 = pd.read_sql(sql1,con)
data1.head()

,user_id,user_id,手机号,FROM_UNIXTIME(oo.pay_time),unionid
0,23562090,23562090,13757488666,2020-05-26 16:00:30,
1,23032948,23032948,18302576536,2020-05-26 16:00:45,
2,24612817,24612817,13764553082,2020-05-26 16:01:01,
3,21548361,21548361,18823129709,2020-05-26 16:02:31,
4,21137382,21137382,17707094907,2020-05-26 16:02:47,


In [24]:
data2 = pd.read_sql(sql2,con)
data2.head()

,user_id,user_id,手机号,FROM_UNIXTIME(oo.pay_time),unionid
0,14537750,14537750,13261585109,2020-03-31 08:41:08,
1,18709706,18709706,15386638323,2020-05-23 17:50:29,
2,18709538,18709538,15124260904,2020-05-23 17:50:47,
3,18711132,18711132,13811975173,2020-05-23 17:51:47,
4,18710570,18710570,13826019785,2020-05-23 17:52:54,


In [25]:
from openpyxl import load_workbook
 
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.
    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]
    Returns: None
    """

# from openpyxl import load_workbook
 
   # import pandas as pd
 
    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')
 
    writer = pd.ExcelWriter(filename, engine='openpyxl')
 
    # Python 2.x: define [FileNotFoundError] exception if it doesn't exist 
    try:
        FileNotFoundError
    except NameError:
        FileNotFoundError = IOError
 
 
    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)
 
        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row
 
        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)
 
        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass
 
    if startrow is None:
        startrow = 0
 
    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)
 
    # save the workbook
    writer.save()
    writer.close()


In [26]:
append_df_to_excel(r"C:\Users\EDZ\Desktop\4点电话.xlsx",data1,sheet_name='支付电话',startcol=0,startrow=1,index=False)

In [27]:
append_df_to_excel(r"C:\Users\EDZ\Desktop\4点电话.xlsx",data2,sheet_name='开班电话',startcol=0,startrow=1,index=False)